In [1]:
from copy import deepcopy

import numpy as np
import torch
import tqdm
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from torch import nn, optim

In [2]:
data = load_iris()
X = data["data"]
y = [[t] for t in data["target"]]
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False).fit(y)
y = ohe.transform(y)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [3]:
xtr, xts, ytr, yts = train_test_split(X, y, train_size=0.7, shuffle=True)

In [4]:
class ClassicDense(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        input, output = layer_sizes[0], layer_sizes[-1]
        self.hidden = nn.ModuleList(
            [
                nn.Linear(layer_sizes[i], layer_sizes[i + 1])
                for i in range(len(layer_sizes) - 2)
            ]
        )
        self.act = nn.ModuleList([nn.ReLU() for i in range(len(layer_sizes) - 2)])
        self.output = nn.Linear(layer_sizes[-2], layer_sizes[-1])

    def forward(self, x):
        for hidden, act in zip(self.hidden, self.act):
            x = act(hidden(x))
        x = self.output(x)
        return x

In [5]:
def get_batched(data, batch_size):
    if batch_size is None:
        return data
    dc = deepcopy(data)
    while dc.numel():
        batch, dc = dc[:batch_size], dc[batch_size:]
        yield batch


def format_stats(loss, acc):
    return f"Loss={loss:.2f}, Accuracy={acc*100:.1f}%"


def train(model, xtr, ytr, batch_size=None):
    cum_loss, cum_acc = [], []
    for xtrb, ytrb in zip(get_batched(xtr, batch_size), get_batched(ytr, batch_size)):
        ypred = model(xtrb)
        loss = loss_fn(ypred, ytrb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (torch.argmax(ypred, 1) == torch.argmax(ytrb, 1)).float().mean()
        cum_loss.append(float(loss))
        cum_acc.append(float(acc))
    return cum_loss, cum_acc


def test(model, xts, yts):
    ypred = model(xts)
    loss = loss_fn(ypred, yts)
    acc = (torch.argmax(ypred, 1) == torch.argmax(xts, 1)).float().mean()
    return float(loss), float(acc)


def train_epochs(
    model, xtr, ytr, xts=None, yts=None, n_epochs=100, batch_size=5, verbose=False
):
    best_acc = -np.inf
    best_weights = None
    train_loss_hist = []
    train_acc_hist = []
    test_loss_hist = []
    test_acc_hist = []

    for epoch in range(n_epochs):
        model.train()
        cum_loss, cum_acc = train(model, xtr, ytr, batch_size)
        model.eval()
        mloss, macc = np.mean(cum_loss), np.mean(cum_acc)
        train_loss_hist.append(mloss)
        train_acc_hist.append(macc)

        if all(x is not None for x in (xts, yts)):
            mloss, macc = test(model, xts, yts)

        if macc > best_acc:
            best_acc = macc
            best_weights = deepcopy(model.state_dict())

        if verbose:
            print(f"Epoch {epoch} validation: {format_stats(mloss, macc)}")
    if verbose:
        print(f"Best validation: {format_stats(mloss, macc)}")
    model.load_state_dict(best_weights)

In [6]:
model = ClassicDense(layer_sizes=[4, 8, 3])
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
pre_loss, pre_acc = test(model, xts, yts)
train_epochs(model, xtr, ytr, xts, yts)
post_loss, post_acc = test(model, xts, yts)
print(format_stats(pre_loss, pre_acc), format_stats(post_loss, post_acc))

model = ClassicDense(layer_sizes=[4, 8, 8, 3])
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
pre_loss, pre_acc = test(model, xts, yts)
train_epochs(model, xtr, ytr, xts, yts)
post_loss, post_acc = test(model, xts, yts)
print(format_stats(pre_loss, pre_acc), format_stats(post_loss, post_acc))

model = ClassicDense(layer_sizes=[4, 8, 16, 8, 3])
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
pre_loss, pre_acc = test(model, xts, yts)
train_epochs(model, xtr, ytr, xts, yts)
post_loss, post_acc = test(model, xts, yts)
print(format_stats(pre_loss, pre_acc), format_stats(post_loss, post_acc))

ClassicDense(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=8, bias=True)
  )
  (act): ModuleList(
    (0): ReLU()
  )
  (output): Linear(in_features=8, out_features=3, bias=True)
)
Loss=1.18, Accuracy=93.3% Loss=1.03, Accuracy=35.6%
ClassicDense(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=8, bias=True)
  )
  (act): ModuleList(
    (0-1): 2 x ReLU()
  )
  (output): Linear(in_features=8, out_features=3, bias=True)
)
Loss=1.06, Accuracy=0.0% Loss=0.91, Accuracy=35.6%
ClassicDense(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
  (act): ModuleList(
    (0-2): 3 x ReLU()
  )
  (output): Linear(in_features=8, out_features=3, bias=True)
)
Loss=1.10, Accuracy=100.0% Loss=1.07, Accuracy=100.0%


In [8]:
model = ClassicDense(layer_sizes=[4, 8, 8, 3])
print(model)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
pre_loss, pre_acc = test(model, xts, yts)
train_epochs(model, xtr, ytr, xts, yts, n_epochs=200, verbose=True)
post_loss, post_acc = test(model, xts, yts)
print(format_stats(pre_loss, pre_acc), format_stats(post_loss, post_acc))

ClassicDense(
  (hidden): ModuleList(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=8, bias=True)
  )
  (act): ModuleList(
    (0-1): 2 x ReLU()
  )
  (output): Linear(in_features=8, out_features=3, bias=True)
)
Epoch 0 validation: Loss=1.10, Accuracy=0.0%
Epoch 1 validation: Loss=1.08, Accuracy=0.0%
Epoch 2 validation: Loss=1.06, Accuracy=0.0%
Epoch 3 validation: Loss=1.04, Accuracy=0.0%
Epoch 4 validation: Loss=1.02, Accuracy=0.0%
Epoch 5 validation: Loss=1.00, Accuracy=0.0%
Epoch 6 validation: Loss=0.98, Accuracy=0.0%
Epoch 7 validation: Loss=0.96, Accuracy=0.0%
Epoch 8 validation: Loss=0.94, Accuracy=0.0%
Epoch 9 validation: Loss=0.93, Accuracy=0.0%
Epoch 10 validation: Loss=0.91, Accuracy=0.0%
Epoch 11 validation: Loss=0.90, Accuracy=0.0%
Epoch 12 validation: Loss=0.88, Accuracy=0.0%
Epoch 13 validation: Loss=0.87, Accuracy=0.0%
Epoch 14 validation: Loss=0.86, Accuracy=0.0%
Epoch 15 validation: Loss=0.85, Accuracy=0.0%
Epoch 